In [1]:
import pandas as pd
import numpy as np
import re
import requests

In [2]:
apartment = pd.read_csv('D:\\0.My data career path\\DA项目课\\总数据.csv', encoding='utf-8')
apartment.columns = ['title', 'link', 'move_in', 'column_a', 'column_b', 
  'summary', 'address', 'rent', 'condition_a', 'condition_b', 'facilities', 
  'community_facility', 'benefits', 'address?']

# 提取卧室，卫浴，可住人数

In [3]:
# 卧室数量
apartment['rooms'] = apartment['condition_a'].apply(lambda x: 
  re.findall(r'卧室[0-9.]个', str(x))[0] if re.findall(r'卧室[0-9.]+个', str(x)) 
  else np.nan).str.extract('(\d+)')
# 卫浴数量
apartment['bathrooms'] = apartment['condition_a'].apply(lambda x: 
  re.findall(r'卫浴[0-9.]个', str(x))[0] if re.findall(r'卫浴[0-9.]+个', str(x)) 
  else np.nan).str.extract('(\d+)')
# 可住人数
apartment['residents'] = apartment['condition_a'].apply(lambda x: 
    re.findall(r'可住[0-9.]人', str(x))[0] if re.findall(r'可住[0-9.]人', str(x)) 
    else np.nan).str.extract('(\d+)')


# 格式化入住时间和是否包含家具（Boolean）

In [4]:
# 可入住时间 （我觉得随时入住，应该设置时间为帖子po出时间，但没有帖子时间，raw data或许有）
apartment['move_in'].drop_duplicates()

def move_in_time(x):
    if str(x).startswith('随时入住'):
        return pd.to_datetime('today').strftime("%Y-%m-%d")
    else:
        return pd.to_datetime(re.findall(r'^[0-9\-]+', str(x))[0]).strftime("%Y-%m-%d")

apartment['start_date'] = apartment['move_in'].apply(lambda x: move_in_time(x))

In [5]:
# 是否包含家具
apartment['furnished'] = apartment['condition_a'].apply(
    lambda x: 1 if re.findall(r'家具包含', str(x)) else 0)

# 距离各个校区的步行距离和坐车距离

In [6]:
Places = apartment['benefits'].apply(lambda x: re.findall(r'(.+?)：', str(x)))

In [7]:
places = set()
for item in Places:
    for it in item:
        i_tmp = it.replace(' ', '')
        i = i_tmp.replace('Campus','校区') # campus和校区一样
        if i and i!='nan': 
            places.add(i)

In [8]:
places

{'悉尼大学',
 '悉尼电影学院',
 '悉尼科技大学',
 '拉筹伯大学悉尼校区',
 '新南威尔士大学',
 '新南威尔士大学Paddington校区',
 '泰勒学院',
 '泰勒语言学校',
 '西悉尼大学',
 '西悉尼大学Campbelltown分校',
 '西悉尼大学Parramatta分校',
 '麦考瑞大学'}

In [9]:
# 提取走路距离数据
def walk_distance(surrounding, s):
    replaced_str = s.replace('\n','')
    end_index = re.search(surrounding,replaced_str).end()
    sub_str = replaced_str[end_index+1:end_index+22]
    if re.search(surrounding,replaced_str)!= None:
        return re.findall(r'步行距离(.+?)M',sub_str)

# 提取坐车距离数据
def bus_distance(surrounding, s):
    replaced_str = s.replace('\n','')
    end_index = re.search(surrounding,replaced_str).end()
    sub_str = replaced_str[end_index+1:end_index+22]
    if re.search(surrounding,replaced_str)!= None:
        return re.findall(r'坐车距离(.+?)M',sub_str)

def get_num(mlist):
    if len(mlist) == 0:
        return np.nan
    else:
        return int(mlist[0])

In [10]:
for place in places:
    apartment[place+'步行距离'] = apartment['benefits'].apply(lambda x: get_num(walk_distance(place,str(x))) if place in str(x) else np.nan)

In [11]:
for place in places:
    apartment[place+'坐车距离'] = apartment['benefits'].apply(lambda x: get_num(bus_distance(place,str(x))) if place in str(x) else np.nan)

# 租金和地址的错位，地址删除多余字段

In [12]:
# 交换错误的address和rent
i = -1
for add in apartment['address']:
    i +=1
    if str(add).find('$') != -1:
        apartment.loc[i, ['address', 'rent']] = apartment.loc[i, ['rent', 'address']].to_numpy()
#         print(apartment.loc[i,['address', 'rent']])

In [13]:
def address_replace(add):
    if str(add).find('$') != -1:
        return ''
    else:
        tmp = str(add).replace('查看地图','')
        return tmp.replace('澳大利亚',',Australia')

apartment['address']= apartment['address'].apply(lambda x: address_replace(x))

In [14]:
# 租金 per week
apartment['rent'] = apartment['move_in'].apply(lambda x: 
    re.findall(r'.*?/周', str(x))[0] if re.findall(r'.*?/周', str(x)) 
    else np.nan).str.extract('(\d+)')

# 设施和周边的明细

In [15]:
apartment.loc[(apartment['condition_b'].apply(
    lambda x: str(x).strip().startswith('公寓设施') if x else False))&(
    apartment['facilities'].apply(
    lambda x: not str(x).startswith('公寓设施') if x else False    
    )), ['condition_b', 'facilities']] = apartment.loc[
    (apartment['condition_b'].apply(lambda x: str(x).strip().startswith('公寓设施') if x else False)) &(apartment['facilities'].apply(
    lambda x: not str(x).startswith('公寓设施') if x else False)
    ), ['facilities', 'condition_b']].values

In [16]:
Facilities = apartment['facilities'].apply(lambda x: str(x).split(' '))

In [17]:
facility = set()
for item in Facilities:
    for it in item:
        if it and it!='nan': 
            facility.add(it.replace('\n', ''))


In [18]:
cols = facility.difference({'公寓周边', '公寓设施'})

In [19]:
for col in cols:
    apartment[col] = apartment['facilities'].apply(lambda x: 1 if x and col in str(x) else 0)

# 删除提取前多余的列

In [20]:
apartment.drop(['move_in', 'column_a','column_b','condition_a','condition_b','address?','benefits','community_facility','facilities','summary'],axis=1,inplace=True)

# 删掉全为空的column

In [21]:
len(apartment.columns)

51

In [22]:
def del_nul_col(col_name):
    ser = pd.isnull(apartment[col_name]).value_counts()
    if len(ser) == 1:
        if ser.index[0] == True:
            apartment.drop([col_name],axis=1,inplace=True)
    
for col in apartment.columns:
    del_nul_col(col)

In [23]:
apartment.columns

Index(['title', 'link', 'address', 'rent', 'rooms', 'bathrooms', 'residents',
       'start_date', 'furnished', '悉尼大学步行距离', '新南威尔士大学Paddington校区步行距离',
       '西悉尼大学Campbelltown分校步行距离', '新南威尔士大学步行距离', '麦考瑞大学步行距离', '悉尼科技大学步行距离',
       '泰勒学院步行距离', '拉筹伯大学悉尼校区步行距离', '泰勒语言学校步行距离', '悉尼电影学院步行距离',
       '西悉尼大学Parramatta分校步行距离', '西悉尼大学步行距离', '悉尼大学坐车距离',
       '新南威尔士大学Paddington校区坐车距离', '新南威尔士大学坐车距离', '麦考瑞大学坐车距离', '悉尼科技大学坐车距离',
       '泰勒学院坐车距离', '泰勒语言学校坐车距离', '悉尼电影学院坐车距离', '西悉尼大学Parramatta分校坐车距离',
       '西悉尼大学坐车距离', 'WIFI', '厨房', '游泳池', '餐馆', '火车站', '冰箱', '公园', '公交站', '电梯',
       '空调', '阳台', '健身房', '包bill', '洗衣机', '电视', '超市', '收起', '近city'],
      dtype='object')

# 导出cleaned文件

In [24]:
apartment.to_csv("D:\\0.My data career path\\DA项目课\\总数据_cleaned.csv")

# Request经纬度

In [25]:
# 由于 Request时间较长，只获取了50个sample
sample_apartment = apartment.sample(50)

In [26]:
def geocode(address):
    base = url = "http://api.map.baidu.com/geocoder?address=" + address + "&output=json&key=f247cdb592eb43ebac6ccd27f796e2d2"
    response = requests.get(base)
    answer = response.json()
    return answer['result']['location']['lng'],answer['result']['location']['lat']

In [27]:
sample_apartment['geocode'] = sample_apartment['address'].apply(lambda x: geocode(str(x)) if x != '' else (0,0))

In [28]:
sample_apartment['geocode']

2706    (116.413384, 39.910925)
2935    (116.413384, 39.910925)
2436    (116.413384, 39.910925)
1740    (116.413384, 39.910925)
1033    (116.462685, 39.924496)
2677    (116.413384, 39.910925)
2340    (116.413384, 39.910925)
2160    (116.413384, 39.910925)
2916    (116.413384, 39.910925)
833     (116.413384, 39.910925)
1488    (116.413384, 39.910925)
2834    (116.413384, 39.910925)
2097    (116.413384, 39.910925)
155     (116.413384, 39.910925)
2869    (116.413384, 39.910925)
332                      (0, 0)
702     (116.413384, 39.910925)
1719    (116.413384, 39.910925)
2317    (116.413384, 39.910925)
2446    (116.413384, 39.910925)
2379    (116.413384, 39.910925)
821     (116.413384, 39.910925)
2020    (116.413384, 39.910925)
1982    (116.413384, 39.910925)
1215    (116.413384, 39.910925)
160     (116.413384, 39.910925)
804     (116.413384, 39.910925)
1509    (116.413384, 39.910925)
1666    (116.413384, 39.910925)
809     (116.413384, 39.910925)
406                      (0, 0)
2825    